# 1导入数据

In [2]:
import re
import jieba
from tqdm import tqdm
import pandas as pd

tqdm.pandas()


In [3]:
Travel_tips1 = pd.read_excel(
    '2018-2019茂名（含自媒体）.xlsx', sheet_name=2)     # 游记攻略
Travel_tips2 = pd.read_excel(
    '2020-2021茂名（含自媒体）.xlsx', sheet_name=2)     # 游记攻略
Travel_tips = pd.concat([Travel_tips1, Travel_tips2], axis=0)  # 游记攻略
Travel_tips=Travel_tips.reset_index(drop=True)
def addstr(s):
    return '旅游攻略-'+str(s)
Travel_tips['语料ID'] = Travel_tips['游记ID'].progress_apply(addstr)
Travel_tips['文本'] = Travel_tips['游记标题'] + '\n'+Travel_tips['正文']
Travel_tips['年份'] = pd.to_datetime(Travel_tips['发布时间']).dt.year
Travel_tips

100%|████████████████████████████████████| 294/294 [00:00<00:00, 294021.31it/s]


,游记ID,城市,游记标题,发布时间,出发时间,出行天数,人物,人均费用,正文,语料ID,文本,年份
0,1001,茂名,漫游中国之三、广东(下)……茂名、惠州、河源、和平、韶关,2018-05-02 07:46,NaN,NaN,NaN,NaN,1、茂名放鸡岛\n放鸡岛 （原名湾舟岛，又名汾洲岛）是位于 广东 省 茂名 市 电白 区博贺...,旅游攻略-1001,漫游中国之三、广东(下)……茂名、惠州、河源、和平、韶关\n1、茂名放鸡岛\n放鸡岛 （原名...,2018
1,1002,茂名,粤西美食| 茂名，心中的美食城（不定时更新）,2018-10-15 22:21,NaN,NaN,NaN,NaN,【茂名在哪】\n我的大 茂名 位于 广东 西部，简称粤西，很多人都不知道 茂名 是哪，所以我...,旅游攻略-1002,粤西美食| 茂名，心中的美食城（不定时更新）\n【茂名在哪】\n我的大 茂名 位于 广东 西...,2018
2,1003,茂名,【茂名】三日游记,2018-11-15 11:38,2018-08-25,3 天,一个人,400RMB,行程总览\n【地点】\n茂名\n\n【时间】\n3天（实际游玩时间约2天半）\n\n【交通】...,旅游攻略-1003,【茂名】三日游记\n行程总览\n【地点】\n茂名\n\n【时间】\n3天（实际游玩时间约2天...,2018
3,1004,茂名,几分钟和你吃遍茂名美食-粤西茂名家乡发展有你我,2018-11-30 18:12,2018-11-30,1 天,和朋友,50RMB,前言\n几分钟真的吃不遍 茂名 ，题目只是一个噱头 🔥🔥这是我第一次写家乡的游记 很多店我觉...,旅游攻略-1004,几分钟和你吃遍茂名美食-粤西茂名家乡发展有你我\n前言\n几分钟真的吃不遍 茂名 ，题目只是...,2018
4,1005,茂名,寻找茂名的历史——周末游高州、化州,2018-12-06 08:09,2018-11-24,2 天,一个人,800RMB,茂名 原本相对于 广州 而言，交通稍差，不适合周末二日游。随着江湛铁路的开通，交通已经不成问...,旅游攻略-1005,寻找茂名的历史——周末游高州、化州\n茂名 原本相对于 广州 而言，交通稍差，不适合周末二日...,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
289,1290,茂名,广东周边游｜御水古温泉｜宝藏度假地,2021-11-10 15:08,NaN,NaN,NaN,NaN,🏨 茂名 御水古温泉度假酒店\n广东 茂名 宝藏小众度假地～\n🌿环境清幽，景色优美～\n🎞...,旅游攻略-1290,广东周边游｜御水古温泉｜宝藏度假地\n🏨 茂名 御水古温泉度假酒店\n广东 茂名 宝藏小众度...,2021
290,1291,茂名,99%的人不知道御水古温泉还可以这样玩！,2021-11-19 13:38,NaN,NaN,NaN,NaN,🉑️骨灰级御水古温泉3日2夜攻略\n🉑️御水古温泉&amp;浪漫海岸度假区\n✅帮你省时又省...,旅游攻略-1291,99%的人不知道御水古温泉还可以这样玩！\n🉑️骨灰级御水古温泉3日2夜攻略\n🉑️御水古温...,2021
291,1292,茂名,高州行一D1佛山至高州：潘洲公园、观山、高州博物馆、宝光塔 、缅茄树、鉴江、瀛洲公园、高凉鼓韵。,2021-12-05 18:14,2021-11-25,1 天,家庭出游,250RMB,2021年11月25日起了一个大早，6:30就在家门口等候网约顺风车。\n晨曦初露，云彩丰富...,旅游攻略-1292,高州行一D1佛山至高州：潘洲公园、观山、高州博物馆、宝光塔 、缅茄树、鉴江、瀛洲公园、高凉鼓...,2021
292,1293,茂名,我们去茂名走马观花（广东⑮）,2021-12-13 16:24,2021-04-12,5 天,家庭出游,500RMB,走马：路上\n爷爷是一个很重男轻女的传统 广东 人，但在看到自己曾孙还没有的情况下，就开始叨...,旅游攻略-1293,我们去茂名走马观花（广东⑮）\n走马：路上\n爷爷是一个很重男轻女的传统 广东 人，但在看到...,2021


# 2分词及去除停用词

In [4]:
##删除特殊符号及文本分词
train_data = Travel_tips
def clearTxt(line):
    if line != '':
        line = str(line).strip()
        #去除文本中的英文和数字
        line = re.sub("[a-zA-Z0-9]", "", line)
        #只保留中文、大小写字母
        reg = "[^0-9A-Za-z\u4e00-\u9fa5]"
        line = re.sub(reg, '', line)
        #分词
        segList = jieba.cut(line, cut_all=False)
        segSentence = ''
        for word in segList:
            if word != '\t':
                segSentence += word + " "
    return segSentence.strip()


text = train_data['文本'].progress_apply(clearTxt)
text


  0%|                                                  | 0/294 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\USER\AppData\Local\Temp\jieba.cache
Loading model cost 1.046 seconds.
Prefix dict has been built successfully.
100%|████████████████████████████████████████| 294/294 [00:04<00:00, 69.47it/s]


0      漫游 中国 之三 广东 下 茂名 惠州 河源 和平 韶关 茂名 放鸡岛 放鸡岛 原名 湾 舟...
1      粤西 美食 茂名 心中 的 美食城 不 定时 更新 茂名 在 哪 我 的 大 茂名 位于 广...
2      茂名 三日 游记 行程 总览 地点 茂名 时间 天 实际 游玩 时间 约 天半 交通 大巴车...
3      几分钟 和 你 吃 遍 茂名 美食 粤西 茂名 家乡 发展 有 你 我 前言 几分钟 真的 ...
4      寻找 茂名 的 历史 周末 游 高州 化州 茂名 原本 相对 于 广州 而言 交通 稍差 不...
                             ...                        
289    广东 周边游 御水古 温泉 宝藏 度假地 茂名 御水古 温泉 度假 酒店 广东 茂名 宝藏 ...
290    的 人 不 知道 御水古 温泉 还 可以 这样 玩 骨灰级 御水古 温泉 日夜 攻略 御水古...
291    高州 行一 佛山 至 高州 潘洲 公园 观山 高州 博物馆 宝 光塔 缅茄树 鉴江 瀛洲 公...
292    我们 去 茂名 走马观花 广东 走 马路上 爷爷 是 一个 很 重男轻女 的 传统 广东 人...
293    高州 行一 深镇 镇耀 新村 昨晚 夜宿 仙人洞 附近 民宿 一 仙人 阁 山庄 山庄 为 ...
Name: 文本, Length: 294, dtype: object

In [5]:
##去除停用词
import codecs
#读取停顿词列表
stopword_list = [k.strip() for k in open(
    'cn_stopwords.txt', encoding='utf8').readlines() if k.strip() != '']
def stopWord(line):
    sentence = delstopword(line, stopword_list)
    return sentence
#删除停用词
def delstopword(line, stopkey):
    wordList = line.split(' ')
    sentence = ''
    for word in wordList:
        word = word.strip()
        if word not in stopkey:
            if word != '\t':
                sentence += word + " "
    return sentence.strip()


train_data['分词后文本'] = text
text2 = list(train_data['分词后文本'].progress_apply(stopWord))
text2


100%|███████████████████████████████████████| 294/294 [00:02<00:00, 102.94it/s]


['漫游 中国 之三 广东 茂名 惠州 河源 和平 韶关 茂名 放鸡岛 放鸡岛 原名 湾 舟岛 又名 汾洲岛 位于 广东省 茂名市 电 白区 博贺镇 东南 公里 岛屿 岛上 最高点 高米 面积 平方公里 茂名市 最大 海岛 惠州 巽 寮 湾 惠州 巽 寮 湾 读 位于 广东省 惠州市 巽 寮 湾 绵延 公里 海滩 沙质 洁白 细幼 形如 白金 海沙 含硅量 达 世界 罕见 海水 更是 青翠 碧绿 原生态 级 水质 更属 国内 罕有 海滩 百米 水深 超过 米 属 全世界 顶级 天然 海水浴场 空气 负氧离子 含量 最高 达个 高于 三亚 国内 顶级 天然 氧吧',
 '粤西 美食 茂名 心中 美食城 定时 更新 茂名 茂名 位于 广东 西部 简称 粤西 很多 知道 茂名 介绍 会 说 湛江 旁离 广西 很近 哈哈哈哈 说 潮汕 美食 聚集地 去过 汕头 发现 粤东 粤西 美食 很多 相似 地方 茂名 东西 好吃 比较 分散 当地人 很难 找到 帖子 其实 外面 太 想念 家里 总结 家乡 美食 解解馋 有缘 见到 当做 广东 美食 科普 贴 美食 早餐 牛腩 粉 早茶 国际 大酒店 熹 龙华 海等 石墨 肠粉 捞粉 猪杂 汤食惯 嘴 点心 籺 发 籺 簸箕 炊 凉粉 甜 豆花 电城 蚝 炸 陈皮 鸡翅 豆芽 粉化 州 糖水 主食 海鲜 陈村 绉 记 化州 香油 鸡 农家乐 四海一家 水东 鸭 粥 水东 芥菜 猪红 麦菜 煲 杂鱼 煲 白切 鸡 水土 鸡 黄鳝 粥 隔 水蒸 鸡 宵夜 烧生 蚝 化州 牛杂 中中 炒 田螺 沙煲 粉 水果 香蕉 荔枝 龙眼 三华 李 黄皮 杨桃 菠萝蜜 早餐 早餐 选择 很多 爸妈 喜欢 早上 一碗 白粥 配 一条 香 煎 海鱼 萝卜干 煎蛋 喜欢 牛腩 粉 肠粉 周末 时间 充足 喝个 早茶 说 起来 茂名 早晨 似乎 挺 悠闲 早茶 广东 喜欢 早茶 小时候 喜欢 一车 车 各式各样 糕点 现在 更 喜欢 一边 喝茶 一边 吃 东西 一边 喝 家人 聊天 感觉 广东 喝 早茶 小时 基本上 午饭 不用 吃 早茶 必点 青 红椒 蒸 凤爪 比较 软烂 非常 好吃 蒸 排骨 清蒸 豉油 鲍汁 凤爪 比较 有嚼头 蒸 凤爪 口感 完全 煎 饺蒸 金钱 肚 虾饺 蛋挞 反正 推车 挑 顺眼 吃 传统 茶楼 推车 不同 推车 不同 食材 蒸点

# 3TF-IDF提取关键词

In [6]:
train_data["提词文本"]=text2

In [7]:
train_data

,游记ID,城市,游记标题,发布时间,出发时间,出行天数,人物,人均费用,正文,语料ID,文本,年份,分词后文本,提词文本
0,1001,茂名,漫游中国之三、广东(下)……茂名、惠州、河源、和平、韶关,2018-05-02 07:46,NaN,NaN,NaN,NaN,1、茂名放鸡岛\n放鸡岛 （原名湾舟岛，又名汾洲岛）是位于 广东 省 茂名 市 电白 区博贺...,旅游攻略-1001,漫游中国之三、广东(下)……茂名、惠州、河源、和平、韶关\n1、茂名放鸡岛\n放鸡岛 （原名...,2018,漫游 中国 之三 广东 下 茂名 惠州 河源 和平 韶关 茂名 放鸡岛 放鸡岛 原名 湾 舟...,漫游 中国 之三 广东 茂名 惠州 河源 和平 韶关 茂名 放鸡岛 放鸡岛 原名 湾 舟岛 ...
1,1002,茂名,粤西美食| 茂名，心中的美食城（不定时更新）,2018-10-15 22:21,NaN,NaN,NaN,NaN,【茂名在哪】\n我的大 茂名 位于 广东 西部，简称粤西，很多人都不知道 茂名 是哪，所以我...,旅游攻略-1002,粤西美食| 茂名，心中的美食城（不定时更新）\n【茂名在哪】\n我的大 茂名 位于 广东 西...,2018,粤西 美食 茂名 心中 的 美食城 不 定时 更新 茂名 在 哪 我 的 大 茂名 位于 广...,粤西 美食 茂名 心中 美食城 定时 更新 茂名 茂名 位于 广东 西部 简称 粤西 很多 ...
2,1003,茂名,【茂名】三日游记,2018-11-15 11:38,2018-08-25,3 天,一个人,400RMB,行程总览\n【地点】\n茂名\n\n【时间】\n3天（实际游玩时间约2天半）\n\n【交通】...,旅游攻略-1003,【茂名】三日游记\n行程总览\n【地点】\n茂名\n\n【时间】\n3天（实际游玩时间约2天...,2018,茂名 三日 游记 行程 总览 地点 茂名 时间 天 实际 游玩 时间 约 天半 交通 大巴车...,茂名 三日 游记 行程 总览 地点 茂名 时间 天 实际 游玩 时间 约 天半 交通 大巴车...
3,1004,茂名,几分钟和你吃遍茂名美食-粤西茂名家乡发展有你我,2018-11-30 18:12,2018-11-30,1 天,和朋友,50RMB,前言\n几分钟真的吃不遍 茂名 ，题目只是一个噱头 🔥🔥这是我第一次写家乡的游记 很多店我觉...,旅游攻略-1004,几分钟和你吃遍茂名美食-粤西茂名家乡发展有你我\n前言\n几分钟真的吃不遍 茂名 ，题目只是...,2018,几分钟 和 你 吃 遍 茂名 美食 粤西 茂名 家乡 发展 有 你 我 前言 几分钟 真的 ...,几分钟 吃 遍 茂名 美食 粤西 茂名 家乡 发展 前言 几分钟 真的 吃 不遍 茂名 题目...
4,1005,茂名,寻找茂名的历史——周末游高州、化州,2018-12-06 08:09,2018-11-24,2 天,一个人,800RMB,茂名 原本相对于 广州 而言，交通稍差，不适合周末二日游。随着江湛铁路的开通，交通已经不成问...,旅游攻略-1005,寻找茂名的历史——周末游高州、化州\n茂名 原本相对于 广州 而言，交通稍差，不适合周末二日...,2018,寻找 茂名 的 历史 周末 游 高州 化州 茂名 原本 相对 于 广州 而言 交通 稍差 不...,寻找 茂名 历史 周末 游 高州 化州 茂名 原本 相对 广州 交通 稍差 适合 周末 二日...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,1290,茂名,广东周边游｜御水古温泉｜宝藏度假地,2021-11-10 15:08,NaN,NaN,NaN,NaN,🏨 茂名 御水古温泉度假酒店\n广东 茂名 宝藏小众度假地～\n🌿环境清幽，景色优美～\n🎞...,旅游攻略-1290,广东周边游｜御水古温泉｜宝藏度假地\n🏨 茂名 御水古温泉度假酒店\n广东 茂名 宝藏小众度...,2021,广东 周边游 御水古 温泉 宝藏 度假地 茂名 御水古 温泉 度假 酒店 广东 茂名 宝藏 ...,广东 周边游 御水古 温泉 宝藏 度假地 茂名 御水古 温泉 度假 酒店 广东 茂名 宝藏 ...
290,1291,茂名,99%的人不知道御水古温泉还可以这样玩！,2021-11-19 13:38,NaN,NaN,NaN,NaN,🉑️骨灰级御水古温泉3日2夜攻略\n🉑️御水古温泉&amp;浪漫海岸度假区\n✅帮你省时又省...,旅游攻略-1291,99%的人不知道御水古温泉还可以这样玩！\n🉑️骨灰级御水古温泉3日2夜攻略\n🉑️御水古温...,2021,的 人 不 知道 御水古 温泉 还 可以 这样 玩 骨灰级 御水古 温泉 日夜 攻略 御水古...,知道 御水古 温泉 玩 骨灰级 御水古 温泉 日夜 攻略 御水古 温泉 浪漫 海岸 度假区 ...
291,1292,茂名,高州行一D1佛山至高州：潘洲公园、观山、高州博物馆、宝光塔 、缅茄树、鉴江、瀛洲公园、高凉鼓韵。,2021-12-05 18:14,2021-11-25,1 天,家庭出游,250RMB,2021年11月25日起了一个大早，6:30就在家门口等候网约顺风车。\n晨曦初露，云彩丰富...,旅游攻略-1292,高州行一D1佛山至高州：潘洲公园、观山、高州博物馆、宝光塔 、缅茄树、鉴江、瀛洲公园、高凉鼓...,2021,高州 行一 佛山 至 高州 潘洲 公园 观山 高州 博物馆 宝 光塔 缅茄树 鉴江 瀛洲 公...,高州 行一 佛山 高州 潘洲 公园 观山 高州 博物馆 宝 光塔 缅茄树 鉴江 瀛洲 公园 ...
292,1293,茂名,我们去茂名走马观花（广东⑮）,2021-12-13 16:24,2021-04-12,5 天,家庭出游,500RMB,走马：路上\n爷爷是一个很重男轻女的传统 广东 人，但在看到自己曾孙还没有的情况下，就开始叨...,旅游攻略-1293,我们去茂名走马观花（广东⑮）\n走马：路上\n爷爷是一个很重男轻女的传统 广东 人，但在看到...,2021,我们 去 茂名 走马观花 广东 走 马路上 爷爷 是 一个 很 重男轻女 的 传统 广东 人...,茂名 走马观花 广东 走 马路上 爷爷 一个 重男轻女 传统 广东 看到 曾孙 没有 情况 ...


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_data["提词文本"])
print(vectorizer.get_feature_names())
print(X.shape)

['一一', '一万', '一上', '一下', '一下下', '一下子', '一世', '一丝', '一丝丝', '一两个', '一两只', '一两天', '一两家', '一两张', '一两次', '一两片', '一两秒', '一两米', '一个', '一个个', '一个劲', '一个半', '一个四岁', '一个团', '一个圈', '一个多', '一个多月', '一个月', '一个样', '一个桶', '一个点', '一个碗', '一个第一次', '一个集', '一中', '一串', '一举', '一举一动', '一九九四年', '一书', '一事', '一亭', '一人', '一人元', '一代', '一代人', '一代代', '一件', '一份', '一众', '一会', '一会儿', '一位', '一住', '一体', '一体化', '一侧', '一個', '一做', '一元复始', '一入', '一共', '一兵一卒', '一出', '一分', '一分钟', '一切办法', '一切都是', '一初', '一到', '一刹', '一刹那', '一刻', '一副', '一副副', '一勺', '一包', '一千三百多年', '一千个', '一千多', '一千多年', '一千多米', '一千年', '一半', '一南', '一博贺', '一卫', '一卷', '一厅', '一厨', '一去', '一双', '一口', '一口价', '一口气', '一句', '一只', '一台', '一叶', '一叶障目', '一号', '一同', '一名', '一周', '一周年', '一味', '一命', '一品', '一回', '一团', '一围', '一圈', '一圩', '一地', '一场', '一坐', '一块', '一块钱', '一堆', '一声', '一声令下', '一声声', '一壶', '一处', '一夜', '一大', '一大堆', '一大家子', '一大批', '一大早', '一大笔', '一大群', '一天', '一头', '一奇', '一套', '一字之差', '一字排开', '一季', '一定', '一室', '一家', '一家之主', '一家亲', '一家人', '一家子', '一家老小', '一寸', '一对', '一对一', '一封', '

In [9]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
X.toarray().shape

(294, 23458)

In [11]:
#生成特征词语
feature_word=[]
for i in vectorizer.get_feature_names():
    feature_word.append(i)

In [12]:
extract={}
for i in range(len(X.toarray())):
    extract[i]=[feature_word[i]
                        for i in X.toarray()[i].argsort()[:-8:-1]]


In [13]:
extract

{0: ['惠州', '顶级', '国内', '天然', '放鸡岛', '白金', '形如'],
 1: ['早茶', '喜欢', '必点', '推车', '凤爪', '排骨', '点菜'],
 2: ['博物馆', '高州', '桂圆', '道菜', '甩卖', '班次', '高铁'],
 3: ['推荐', '人均', '营销', '家乡', '这家', '五折', '黑卡'],
 4: ['高州', '茂名', '高州县', '治所', '电白', '电白县', '郭县'],
 5: ['演讲', '代表', '巫帅', '电视台', '抽签', '孙波', '签到'],
 6: ['狗爷', '村里', '村子', '虎子', '木生', '没人', '几条'],
 7: ['老姐', '放鸡岛', '太阳', '景色', '排队', '布丁', '看到'],
 8: ['丹霞山', '水洞', '石花', '龙虎山', '天涯海角', '广垦', '热带'],
 9: ['活下去', '台下', '掌声', '药师', '医生', '主持人', '常去'],
 10: ['孩子', '海岸', '浪漫', '庭院', '海陵', '贝壳', '温德姆'],
 11: ['碧桂园', '凤凰', '酒店', '小宇', '茂名', '兴记', '老字号'],
 12: ['私房', '三月', '红珊瑚', '试菜', '菠萝蜜', '得福', '田心'],
 13: ['执裁', '纠结', '安踏', '篮球', '绿树', '临场', '从左到右'],
 14: ['小树', '复联', '屋顶', '插播', '没能', '树林', '一堆'],
 15: ['巴士', '车费', '一滩', '放鸡岛', '直达', '比较', '岛上'],
 16: ['饭局', '贡园', '执裁', '品味', '桃花', '喝酒', '安踏'],
 17: ['沉香', '种植', '君元', '公司', '有限公司', '生产', '饮片'],
 18: ['执裁', '贡园', '安踏', '学垌', '企业', '红荔', '联赛'],
 19: ['期待', '生活', '幸福', '心里', '有事', '状态', '事情'],
 20: ['露

In [63]:
a=pd.DataFrame(extract).T
a.columns=["第一关键词","第二关键词","第三关键词"]
a.to_excel("关键词.xlsx")

In [17]:
train_data["产品名称"]=train_data["人物"]

In [18]:
for i in range(len(extract)):
    train_data["产品名称"][i]=extract[i][0]

<ipython-input-18-4eed539d8f7a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["产品名称"][i]=extract[i][0]


In [19]:
train_data

,游记ID,城市,游记标题,发布时间,出发时间,出行天数,人物,人均费用,正文,语料ID,文本,年份,分词后文本,提词文本,产品名称
0,1001,茂名,漫游中国之三、广东(下)……茂名、惠州、河源、和平、韶关,2018-05-02 07:46,NaN,NaN,NaN,NaN,1、茂名放鸡岛\n放鸡岛 （原名湾舟岛，又名汾洲岛）是位于 广东 省 茂名 市 电白 区博贺...,旅游攻略-1001,漫游中国之三、广东(下)……茂名、惠州、河源、和平、韶关\n1、茂名放鸡岛\n放鸡岛 （原名...,2018,漫游 中国 之三 广东 下 茂名 惠州 河源 和平 韶关 茂名 放鸡岛 放鸡岛 原名 湾 舟...,漫游 中国 之三 广东 茂名 惠州 河源 和平 韶关 茂名 放鸡岛 放鸡岛 原名 湾 舟岛 ...,惠州
1,1002,茂名,粤西美食| 茂名，心中的美食城（不定时更新）,2018-10-15 22:21,NaN,NaN,NaN,NaN,【茂名在哪】\n我的大 茂名 位于 广东 西部，简称粤西，很多人都不知道 茂名 是哪，所以我...,旅游攻略-1002,粤西美食| 茂名，心中的美食城（不定时更新）\n【茂名在哪】\n我的大 茂名 位于 广东 西...,2018,粤西 美食 茂名 心中 的 美食城 不 定时 更新 茂名 在 哪 我 的 大 茂名 位于 广...,粤西 美食 茂名 心中 美食城 定时 更新 茂名 茂名 位于 广东 西部 简称 粤西 很多 ...,早茶
2,1003,茂名,【茂名】三日游记,2018-11-15 11:38,2018-08-25,3 天,一个人,400RMB,行程总览\n【地点】\n茂名\n\n【时间】\n3天（实际游玩时间约2天半）\n\n【交通】...,旅游攻略-1003,【茂名】三日游记\n行程总览\n【地点】\n茂名\n\n【时间】\n3天（实际游玩时间约2天...,2018,茂名 三日 游记 行程 总览 地点 茂名 时间 天 实际 游玩 时间 约 天半 交通 大巴车...,茂名 三日 游记 行程 总览 地点 茂名 时间 天 实际 游玩 时间 约 天半 交通 大巴车...,博物馆
3,1004,茂名,几分钟和你吃遍茂名美食-粤西茂名家乡发展有你我,2018-11-30 18:12,2018-11-30,1 天,和朋友,50RMB,前言\n几分钟真的吃不遍 茂名 ，题目只是一个噱头 🔥🔥这是我第一次写家乡的游记 很多店我觉...,旅游攻略-1004,几分钟和你吃遍茂名美食-粤西茂名家乡发展有你我\n前言\n几分钟真的吃不遍 茂名 ，题目只是...,2018,几分钟 和 你 吃 遍 茂名 美食 粤西 茂名 家乡 发展 有 你 我 前言 几分钟 真的 ...,几分钟 吃 遍 茂名 美食 粤西 茂名 家乡 发展 前言 几分钟 真的 吃 不遍 茂名 题目...,推荐
4,1005,茂名,寻找茂名的历史——周末游高州、化州,2018-12-06 08:09,2018-11-24,2 天,一个人,800RMB,茂名 原本相对于 广州 而言，交通稍差，不适合周末二日游。随着江湛铁路的开通，交通已经不成问...,旅游攻略-1005,寻找茂名的历史——周末游高州、化州\n茂名 原本相对于 广州 而言，交通稍差，不适合周末二日...,2018,寻找 茂名 的 历史 周末 游 高州 化州 茂名 原本 相对 于 广州 而言 交通 稍差 不...,寻找 茂名 历史 周末 游 高州 化州 茂名 原本 相对 广州 交通 稍差 适合 周末 二日...,高州
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,1290,茂名,广东周边游｜御水古温泉｜宝藏度假地,2021-11-10 15:08,NaN,NaN,NaN,NaN,🏨 茂名 御水古温泉度假酒店\n广东 茂名 宝藏小众度假地～\n🌿环境清幽，景色优美～\n🎞...,旅游攻略-1290,广东周边游｜御水古温泉｜宝藏度假地\n🏨 茂名 御水古温泉度假酒店\n广东 茂名 宝藏小众度...,2021,广东 周边游 御水古 温泉 宝藏 度假地 茂名 御水古 温泉 度假 酒店 广东 茂名 宝藏 ...,广东 周边游 御水古 温泉 宝藏 度假地 茂名 御水古 温泉 度假 酒店 广东 茂名 宝藏 ...,温泉
290,1291,茂名,99%的人不知道御水古温泉还可以这样玩！,2021-11-19 13:38,NaN,NaN,NaN,NaN,🉑️骨灰级御水古温泉3日2夜攻略\n🉑️御水古温泉&amp;浪漫海岸度假区\n✅帮你省时又省...,旅游攻略-1291,99%的人不知道御水古温泉还可以这样玩！\n🉑️骨灰级御水古温泉3日2夜攻略\n🉑️御水古温...,2021,的 人 不 知道 御水古 温泉 还 可以 这样 玩 骨灰级 御水古 温泉 日夜 攻略 御水古...,知道 御水古 温泉 玩 骨灰级 御水古 温泉 日夜 攻略 御水古 温泉 浪漫 海岸 度假区 ...,温泉
291,1292,茂名,高州行一D1佛山至高州：潘洲公园、观山、高州博物馆、宝光塔 、缅茄树、鉴江、瀛洲公园、高凉鼓韵。,2021-12-05 18:14,2021-11-25,1 天,家庭出游,250RMB,2021年11月25日起了一个大早，6:30就在家门口等候网约顺风车。\n晨曦初露，云彩丰富...,旅游攻略-1292,高州行一D1佛山至高州：潘洲公园、观山、高州博物馆、宝光塔 、缅茄树、鉴江、瀛洲公园、高凉鼓...,2021,高州 行一 佛山 至 高州 潘洲 公园 观山 高州 博物馆 宝 光塔 缅茄树 鉴江 瀛洲 公...,高州 行一 佛山 高州 潘洲 公园 观山 高州 博物馆 宝 光塔 缅茄树 鉴江 瀛洲 公园 ...,观山
292,1293,茂名,我们去茂名走马观花（广东⑮）,2021-12-13 16:24,2021-04-12,5 天,家庭出游,500RMB,走马：路上\n爷爷是一个很重男轻女的传统 广东 人，但在看到自己曾孙还没有的情况下，就开始叨...,旅游攻略-1293,我们去茂名走马观花（广东⑮）\n走马：路上\n爷爷是一个很重男轻女的传统 广东 人，但在看到...,2021,我们 去 茂名 走马观花 广东 走 马路上 爷爷 是 一个 很 重男轻女 的 传统 广东 人...,茂名 走马观花 广东 走 马路上 爷爷 一个 重男轻女 传统 广东 看到 曾孙 没有 情况 ...,文和友


# 4汇总其他数据

In [64]:
Travel_tips=train_data
Hotel_reviews1 = pd.read_excel(
    '2018-2019茂名（含自媒体）.xlsx', sheet_name=0)   # 酒店评论
Scenic_reviews1 = pd.read_excel(
    '2018-2019茂名（含自媒体）.xlsx', sheet_name=1)  # 景区评论
Dining_reviews1 = pd.read_excel(
    '2018-2019茂名（含自媒体）.xlsx', sheet_name=3)  # 餐饮评论
Hotel_reviews2 = pd.read_excel(
    '2020-2021茂名（含自媒体）.xlsx', sheet_name=0)   # 酒店评论
Scenic_reviews2 = pd.read_excel(
    '2020-2021茂名（含自媒体）.xlsx', sheet_name=1)  # 景区评论
Dining_reviews2 = pd.read_excel(
    '2020-2021茂名（含自媒体）.xlsx', sheet_name=3)  # 餐饮评论
Hotel_reviews = pd.concat([Hotel_reviews1, Hotel_reviews2],axis=0)  # 酒店评论
Scenic_reviews = pd.concat([Scenic_reviews1, Scenic_reviews2], axis=0)  # 景区评论
Dining_reviews = pd.concat([Dining_reviews1, Dining_reviews2], axis=0)  # 餐饮评论

In [65]:
def addstr(s):
    return '景区评论-'+str(s)

Scenic_reviews['语料ID'] = Scenic_reviews['景区评论ID'].progress_apply(addstr)
Scenic_reviews['文本'] = Scenic_reviews['评论内容']
Scenic_reviews['产品名称'] = Scenic_reviews['景区名称']
Scenic_reviews['年份'] = pd.to_datetime(Scenic_reviews['评论日期']).dt.year

def addstr(s):
    return '酒店评论-'+str(s)

Hotel_reviews['语料ID'] = Hotel_reviews['酒店评论ID'].progress_apply(addstr)
Hotel_reviews['文本'] = Hotel_reviews['评论内容']
Hotel_reviews['产品名称'] = Hotel_reviews['酒店名称']
Hotel_reviews['年份'] = pd.to_datetime(Hotel_reviews['评论日期']).dt.year

def addstr(s):
    return '餐饮评论-'+str(s)


Dining_reviews['语料ID'] = Dining_reviews['餐饮评论ID'].progress_apply(addstr)
Dining_reviews['文本'] = Dining_reviews['评论内容'] + '\n'+Dining_reviews['标题']
Dining_reviews['产品名称'] = Dining_reviews['餐饮名称']
Dining_reviews['年份'] = pd.to_datetime(Dining_reviews['评论日期']).dt.year


100%|██████████████████████████████████| 6984/6984 [00:00<00:00, 104233.01it/s]


# 5汇总全部数据

In [67]:

all_df = pd.DataFrame(columns=['语料ID', '文本', '产品名称'])
all_df['语料ID'] = pd.concat([Dining_reviews['语料ID'], Hotel_reviews['语料ID'],
                           Scenic_reviews['语料ID'], Travel_tips['语料ID']], axis=0)
all_df['产品名称'] = pd.concat([Dining_reviews['产品名称'], Hotel_reviews['产品名称'],
                           Scenic_reviews['产品名称'], Travel_tips['产品名称']], axis=0)
all_df['文本'] = pd.concat([Dining_reviews['文本'], Hotel_reviews['文本'],
                         Scenic_reviews['文本'], Travel_tips['文本']], axis=0)
all_df['年份'] = pd.concat([Dining_reviews['年份'], Hotel_reviews['年份'],
                         Scenic_reviews['年份'], Travel_tips['年份']], axis=0)

all_df=all_df.reset_index(drop=True)
all_df.head()

,语料ID,文本,产品名称,年份
0,餐饮评论-1001,很好吃推荐！\n主食3选1，有赠品,盛香烧鹅（东方市场店）,2018
1,餐饮评论-1002,超级好吃 老板又好 量又多\n水果忌廉夹心蛋糕（二层）1个，约4磅，圆,清香面包店（车田街店）,2018
2,餐饮评论-1003,好吃，不得不说比门口正对面那家好吃。服务态度也好。炸的皮松软，不会炸得很硬。\n小吃6选1，...,功夫鸡排（光华北路店）,2018
3,餐饮评论-1004,品种少，冷冻食品多。 7点就不再出食物，难以接受。\n自助晚餐（浪漫海岸跨年音乐节）,茂名浪漫海岸温德姆酒店望海餐厅,2018
4,餐饮评论-1005,味道不错，至少吃起来不腻，并且全家7个人吃还是挺多的了，就比较实惠咯\n水果忌廉夹心蛋糕（二...,清香面包店（车田街店）,2018


In [23]:
product_id = ['ID'+str(i+1) for i in range(len(all_df))]
all_df['产品ID'] = product_id
result2 = all_df[['语料ID','产品ID','产品名称']]
result2


,语料ID,产品ID,产品名称
0,餐饮评论-1001,ID1,盛香烧鹅（东方市场店）
1,餐饮评论-1002,ID2,清香面包店（车田街店）
2,餐饮评论-1003,ID3,功夫鸡排（光华北路店）
3,餐饮评论-1004,ID4,茂名浪漫海岸温德姆酒店望海餐厅
4,餐饮评论-1005,ID5,清香面包店（车田街店）
...,...,...,...
9569,旅游攻略-1290,ID9570,温泉
9570,旅游攻略-1291,ID9571,温泉
9571,旅游攻略-1292,ID9572,观山
9572,旅游攻略-1293,ID9573,文和友


# 6导出数据，最终定下来再导出

In [114]:
result2.to_csv('result2-1.csv', index=False)
all_df.to_csv('问题二所有数据汇总.csv', index=False)
